# Kerakli kutubxonalar yuklanadi

In [5]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import tensorflow_datasets as tfds

2023-08-08 10:38:50.243698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.13.0


# Datasetni yuklash

In [6]:
dataset = tfds.load("stanford_online_products", split = "train")
batch_size = 32
def preprocess_data(example):
    image = example['image']
    image = tf.image.resize(image, [50, 50])
    image = tf.cast(image, tf.float32) / 255.0
    label = example['super_class_id']
    return image, label

dataset = dataset.map(preprocess_data)

dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

2023-08-08 10:38:51.859938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-08 10:38:51.860154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-08 10:38:51.862362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
ds_test = tfds.load('stanford_online_products' , split = 'test')
ds_test = ds_test.map(preprocess_data)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Arxitektura

In [8]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(12)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Uning loss va accuracy parametrlari

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# Train funksiya

In [10]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# Test funksiya

In [11]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# Train jarayoni

In [12]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in dataset:
    train_step(images, labels)
  for images, labels in ds_test:
      test_step(images, labels)


  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

2023-08-08 10:38:52.878454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-08-08 10:38:53.287432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-08 10:38:53.356112: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1d4e1134e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 10:38:53.356154: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-08-08 10:38:53.356158: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-08-08 10:38:53.359815: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 

Epoch 1, Loss: 2.095576524734497, Accuracy: 30.620813369750977, Test Loss: 2.010236978530884, Test Accuracy: 33.87491226196289
Epoch 2, Loss: 1.7002888917922974, Accuracy: 44.393882751464844, Test Loss: 2.0292835235595703, Test Accuracy: 35.140987396240234
Epoch 3, Loss: 1.4255869388580322, Accuracy: 53.63302230834961, Test Loss: 2.2239348888397217, Test Accuracy: 33.50798034667969
Epoch 4, Loss: 1.1535989046096802, Accuracy: 62.549747467041016, Test Loss: 2.504258871078491, Test Accuracy: 32.22703170776367
Epoch 5, Loss: 0.8839621543884277, Accuracy: 71.67637634277344, Test Loss: 3.0003206729888916, Test Accuracy: 29.542823791503906
